# NFT - Etherscan - Google BigQuery - ML - Comitê Classificação

## Packages and libraries

In [1]:
pip install google-cloud-bigquery db-dtypes pandas tensorflow scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
# importar pacotes
import os
import requests
import numpy as np
import glob
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.cloud import bigquery
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.svm import OneClassSVM
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.cluster import DBSCAN

from tensorflow import keras
from tensorflow.keras import layers, models

from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler


In [3]:
# Caminho para o arquivo de credenciais GCP (Google Big Query)
credentials_file = 'Z:/Python/Data/GCP/tough-gearing-363721-6c1a59db780d.json'

# Verifique se o arquivo de credenciais existe
if os.path.isfile(credentials_file):
    print("O arquivo de credenciais existe.")
else:
    print("O arquivo de credenciais não foi encontrado.")

O arquivo de credenciais existe.


In [4]:
try:
    # Tente criar um cliente BigQuery usando as credenciais
    client = bigquery.Client.from_service_account_json(credentials_file)
    print("Autenticação bem-sucedida.")
except Exception as e:
    print(f"Erro de autenticação: {str(e)}")

Autenticação bem-sucedida.


In [5]:
try:
    # Tente criar um cliente BigQuery usando as credenciais
    client = bigquery.Client.from_service_account_json(credentials_file)
    print("Autenticação bem-sucedida.")

    # Use a função dir() para listar atributos e métodos do objeto client
    print("Atributos e métodos disponíveis no objeto client:")
#   print(dir(client))
except Exception as e:
    print(f"Erro de autenticação: {str(e)}")

Autenticação bem-sucedida.
Atributos e métodos disponíveis no objeto client:


In [6]:
# Chave Privada API Etherscan
api_key = "Informe a sua Chave Privada API Etherscan"

In [7]:
# Função para verificar o número de transações de uma NFT
def verificar_transacoes_nft(nft_hash):
    
    # URL da API do Etherscan para obter o histórico de transações de um endereço
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={nft_hash}&apikey={api_key}"

    try:
        # Solicitação à API do Etherscan
        response = requests.get(url)
        data = response.json()

        # Verificação se a solicitação foi bem-sucedida
        if data["status"] == "1":
            # Obter o número de transações
            num_transactions = len(data["result"])
            
            # Verificar se o número de transações é maior que 1500
            if num_transactions > 1500:
                print(f"A NFT possui mais de 1500 transações. Total de transações: {num_transactions}")
            else:
                print(f"A NFT possui {num_transactions} transações, que não atendem ao critério de mais de 1500.")
            return num_transactions
        else:
            print("Ocorreu um erro na solicitação à API do Etherscan.")
    except Exception as e:
        print(f"Erro ao fazer a solicitação à API do Etherscan: {str(e)}")

In [8]:
# Função para executar a consulta no BigQuery e retornar um DataFrame
def executar_consulta_bigquery(nft_hash):

    # SQL - Consulta com o valor dinâmico de nft_hash
    query = f"""
    SELECT
      A.BLOCK_TIMESTAMP,
      A.FROM_ADDRESS,
      A.TO_ADDRESS,
      A.VALUE,
      A.TRANSACTION_HASH,
      B.NONCE,
      B.FROM_ADDRESS AS FROM_ADDRESS_BLOCKCHAIN,
      B.TO_ADDRESS AS TO_ADDRESS_BLOCKCHAIN,
      B.GAS,
      B.RECEIPT_GAS_USED
    FROM
      `bigquery-public-data.crypto_ethereum.token_transfers` AS A
    INNER JOIN
      `bigquery-public-data.crypto_ethereum.transactions` AS B
    ON
      A.transaction_hash = B.HASH
    WHERE
      A.TOKEN_ADDRESS = '{nft_hash}'
      AND A.BLOCK_TIMESTAMP >= (
      SELECT
        MIN(block_timestamp)
      FROM
        `bigquery-public-data.crypto_ethereum.contracts`
      WHERE
        address = '{nft_hash}')
    ORDER BY
      A.BLOCK_TIMESTAMP
    """

    # Execução da consulta SQL
    query_job = client.query(query)

    # Extrair os resultados como uma lista de dicionários
    results = []
    for row in query_job:
        results.append(dict(row.items()))

    # Criar um DataFrame a partir dos resultados
    # import pandas as pd
    result_df = pd.DataFrame(results)

    return result_df

In [ ]:
# Solicitar hash da NFT a ser consultada
nft_hash = input("Digite o hash da NFT que você deseja verificar: ").lower()

# Chamar a função para verificar a quantidade de transações de NFT (Limite definido como superior a 1500 transações)
num_transactions = verificar_transacoes_nft(nft_hash)

# Garante que o valor retornado seja convertido para int
qtde_transactions = int(num_transactions)

if qtde_transactions > 1500:
    print(f"Realizando coleta de transações da NFT {nft_hash} na GCP.")

    # Chamar a função para executar a consulta BigQuery e obter o DataFrame resultante
    resultado_df = executar_consulta_bigquery(nft_hash)

    print(resultado_df.info())

In [ ]:
resultado_df.head()

## Consulta realizada no Google BigQuery para seleção de transações de NFT's 

*  SELECT
  A.BLOCK_TIMESTAMP,
  A.FROM_ADDRESS,
  A.TO_ADDRESS,
  A.VALUE,
  A.TRANSACTION_HASH,
  B.NONCE,
  B.FROM_ADDRESS AS FROM_ADDRESS_BLOCKCHAIN,
  B.TO_ADDRESS AS TO_ADDRESS_BLOCKCHAIN,
  B.GAS,
  B.RECEIPT_GAS_USED
FROM
  `bigquery-public-data.crypto_ethereum.token_transfers` AS A
INNER JOIN
  `bigquery-public-data.crypto_ethereum.transactions` AS B
ON
  A.transaction_hash = B.HASH
WHERE
  A.TOKEN_ADDRESS = `#hash_nft_comum`
  AND A.BLOCK_TIMESTAMP >= (
  SELECT
    MIN(block_timestamp)
  FROM
    `bigquery-public-data.crypto_ethereum.contracts`
  WHERE
    address = `#hash_nft_comum`)
ORDER BY
  A.BLOCK_TIMESTAMP

## Conjunto de Dados - Data Sets (NFT's Blue Chip's e NFT's Comuns)

Arquivos CSV criados a partir da extração de dados no Google BigQuery contém as seguintes informações:

* BLOCK_TIMESTAMP: timestamp do bloco em que a transferência foi registrada.
* FROM_ADDRESS: endereço da carteira que enviou os tokens.
* TO_ADDRESS: endereço da carteira que recebeu os tokens.
* VALUE: valor da transferência em wei, a menor unidade de ether (a criptomoeda da Ethereum).
* TRANSACTION_HASH: hash da transação na blockchain Ethereum.
* NONCE: número de sequência da transação na carteira que a criou.
* FROM_ADDRESS_BLOCKCHAIN: endereço da carteira que enviou a transação na blockchain Ethereum.
* TO_ADDRESS_BLOCKCHAIN: endereço da carteira que recebeu a transação na blockchain Ethereum.
* GAS: quantidade de gas (unidade de custo de processamento na Ethereum) usada pela transação.
* RECEIPT_GAS_USED: quantidade de gas usada para executar a transferência de tokens.

In [9]:
def lista_df():
  # Lista vazia para armazenar os dataframes
  df_list = []

  # Localizar todos os arquivos CSV no diretório atual
  for filename in glob.glob("Z:/Python/Data/Blue_Chips/*.csv"):
      # Ler o arquivo CSV em um dataframe
      df = pd.read_csv(filename)
      # Adicionar o dataframe à lista
      df_list.append(df)

  return df_list

In [10]:
def lista_comum_df():
  # lista vazia para armazenar os dataframes
  df_comum_list = []

  # Localizar todos os arquivos CSV no diretório atual
  for filename in glob.glob("Z:/Python/Data/Comuns/Teste3/*.csv"):

      # Ler o arquivo CSV em um dataframe
      df = pd.read_csv(filename)
      # Adicionar o dataframe à lista
      df_comum_list.append(df)

  return df_comum_list

In [11]:
def process_dataframe(df):
    # Convertendo a coluna 'BLOCK_TIMESTAMP' para o tipo datetime e criando uma nova coluna, do tipo datetime, 'BLOCK_DATE' (data sem o horario dos blocos)
    df['BLOCK_TIMESTAMP'] = pd.to_datetime(df['BLOCK_TIMESTAMP'])

    # Criando uma nova coluna com a data (sem o horário) dos blocos
    df['BLOCK_DATE'] = df['BLOCK_TIMESTAMP'].dt.date

    # Convertendo a coluna 'BLOCK_DATE' para o tipo datetime
    df['BLOCK_DATE'] = pd.to_datetime(df['BLOCK_DATE'])

In [12]:
list_df_blue = []
list_df_blue = lista_df()
len(list_df_blue)

20

In [13]:
# list_df_comum = []
# list_df_comum = lista_comum_df()
# len(list_df_comum)

In [14]:
list_df_comum = []

# Caminho para o arquivo CSV que deseja ler
caminho_arquivo_csv = 'Z:/Python/Data/NFT_saida.csv'

# Utilizar função read_csv para ler o arquivo CSV e criar um DataFrame
resultado_df = pd.read_csv(caminho_arquivo_csv)

list_df_comum.append(resultado_df)

len(list_df_comum)

1

In [15]:
# Loop para iterar sobre as list_df_blue, list_df_comum
for idx, df_group in enumerate([list_df_blue, list_df_comum]):
    if idx == 0:
        group_label = "Blue Chip"
    else:
        group_label = "Comum"

    for i, df_aux in enumerate(df_group):
        print(f"\nData Frame {group_label} {i+1}")
        print("Informações:")
        df_aux.info()
        print("\n5 primeiros registros:")
        print(df_aux.head())



Data Frame Blue Chip 1
Informações:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98206 entries, 0 to 98205
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ADDRESS_BLUE             98206 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     98206 non-null  object
 2   BLOCK_TIMESTAMP          98206 non-null  object
 3   FROM_ADDRESS             98206 non-null  object
 4   TO_ADDRESS               98206 non-null  object
 5   VALUE                    98206 non-null  int64 
 6   TRANSACTION_HASH         98206 non-null  object
 7   NONCE                    98206 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  98206 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    98206 non-null  object
 10  GAS                      98206 non-null  int64 
 11  RECEIPT_GAS_USED         98206 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 9.0+ MB

5 primeiros registros:
                                 ADDRE

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93723 entries, 0 to 93722
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ADDRESS_BLUE             93723 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     93723 non-null  object
 2   BLOCK_TIMESTAMP          93723 non-null  object
 3   FROM_ADDRESS             93723 non-null  object
 4   TO_ADDRESS               93723 non-null  object
 5   VALUE                    93723 non-null  int64 
 6   TRANSACTION_HASH         93723 non-null  object
 7   NONCE                    93723 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  93723 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    93723 non-null  object
 10  GAS                      93723 non-null  int64 
 11  RECEIPT_GAS_USED         93723 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 8.6+ MB

5 primeiros registros:
                                 ADDRESS_BLUE     BLOCK_TIMESTAMP_BLUE  \
0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59662 entries, 0 to 59661
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ADDRESS_BLUE             59662 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     59662 non-null  object
 2   BLOCK_TIMESTAMP          59662 non-null  object
 3   FROM_ADDRESS             59662 non-null  object
 4   TO_ADDRESS               59662 non-null  object
 5   VALUE                    59662 non-null  int64 
 6   TRANSACTION_HASH         59662 non-null  object
 7   NONCE                    59662 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  59662 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    59662 non-null  object
 10  GAS                      59662 non-null  int64 
 11  RECEIPT_GAS_USED         59662 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 5.5+ MB

5 primeiros registros:
                                 ADDRESS_BLUE     BLOCK_TIMESTAMP_BLUE  \
0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210339 entries, 0 to 210338
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   ADDRESS_BLUE             210339 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     210339 non-null  object
 2   BLOCK_TIMESTAMP          210339 non-null  object
 3   FROM_ADDRESS             210339 non-null  object
 4   TO_ADDRESS               210339 non-null  object
 5   VALUE                    210339 non-null  int64 
 6   TRANSACTION_HASH         210339 non-null  object
 7   NONCE                    210339 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  210339 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    210339 non-null  object
 10  GAS                      210339 non-null  int64 
 11  RECEIPT_GAS_USED         210339 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 19.3+ MB

5 primeiros registros:
                                 ADDRESS_BLUE     BLOCK_TI

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98937 entries, 0 to 98936
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ADDRESS_BLUE             98937 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     98937 non-null  object
 2   BLOCK_TIMESTAMP          98937 non-null  object
 3   FROM_ADDRESS             98937 non-null  object
 4   TO_ADDRESS               98937 non-null  object
 5   VALUE                    98937 non-null  int64 
 6   TRANSACTION_HASH         98937 non-null  object
 7   NONCE                    98937 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  98937 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    98937 non-null  object
 10  GAS                      98937 non-null  int64 
 11  RECEIPT_GAS_USED         98937 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 9.1+ MB

5 primeiros registros:
                                 ADDRESS_BLUE     BLOCK_TIMESTAMP_BLUE  \
0

In [16]:
#Preparação das colunas dos dataframes 

# Processando a lista de DataFrames list_df_blue
df_list_blue = []
for df_blue in list_df_blue:
    process_dataframe(df_blue)
    df_list_blue.append(df_blue)

# Processando a lista de DataFrames list_df_comum
df_list_comum = []
for df_comum in list_df_comum:
    process_dataframe(df_comum)
    df_list_comum.append(df_comum)


In [17]:
list_grouped_df_blue = []
list_grouped_df_comum = []

# Loop para agrupar os DataFrames e criar as listas list_grouped_df_blue e list_grouped_df_comum
for dataframe_list, grouped_list in zip([df_list_blue, df_list_comum], [list_grouped_df_blue, list_grouped_df_comum]):
    for df in dataframe_list:
        grouped_df = df.groupby('BLOCK_DATE')
        grouped_list.append(grouped_df)

# Loop para imprimir os tamanhos dos grupos para ambos os DataFrames agrupados
for label, grouped_list in [("Blue Chip", list_grouped_df_blue), ("Comum", list_grouped_df_comum)]:
    print(f"\n{label} DataFrames:")
    
    for i, grouped_df in enumerate(grouped_list):
        print(f"\ngrouped_df_{label} {i+1}")
        
        # Use a função size() para obter o tamanho de cada grupo
        group_sizes = grouped_df.size()
        
        # Imprima os tamanhos dos grupos
        print(group_sizes)



Blue Chip DataFrames:

grouped_df_Blue Chip 1
BLOCK_DATE
2022-01-06    8231
2022-01-07    2905
2022-01-08    1250
2022-01-09     618
2022-01-10     315
              ... 
2023-03-10     102
2023-03-11      36
2023-03-12     150
2023-03-13      64
2023-03-14       5
Length: 433, dtype: int64

grouped_df_Blue Chip 2
BLOCK_DATE
2022-04-29     15000
2022-04-30        81
2022-05-01    131388
2022-05-02     16965
2022-05-03      6447
               ...  
2023-03-10      1182
2023-03-11      1057
2023-03-12       723
2023-03-13       857
2023-03-14        48
Length: 320, dtype: int64

grouped_df_Blue Chip 3
BLOCK_DATE
2021-07-22    9654
2021-07-23     515
2021-07-24     232
2021-07-25     256
2021-07-26     166
              ... 
2023-03-10     206
2023-03-11     152
2023-03-12     101
2023-03-13     174
2023-03-14      22
Length: 601, dtype: int64

grouped_df_Blue Chip 4
BLOCK_DATE
2022-10-31    3734
2022-11-01     165
2022-11-02      94
2022-11-03     110
2022-11-04      20
              .

In [18]:
ts_nft_comum = []  # Lista para armazenar os DataFrames criados a partir de list_grouped_df_comum
ts_nft_blue = []   # Lista para armazenar os DataFrames criados a partir de list_grouped_df_blue

# Loop para iterar sobre list_grouped_df_comum e list_grouped_df_blue
for label, grouped_list, ts_nft_list in [("Comum", list_grouped_df_comum, ts_nft_comum), ("Blue Chip", list_grouped_df_blue, ts_nft_blue)]:
    print(f"\n{label} DataFrames:")
    
    for i, ts in enumerate(grouped_list):
        print(f"\ngrouped_df_{label} {i+1}")
        
        # Criar um novo DataFrame vazio para cada grupo
        df = pd.DataFrame()
        
        # Adicionar a coluna 'BLOCK_DATE' ao DataFrame
        df['BLOCK_DATE'] = ts['BLOCK_DATE'].mean()
        
        # Número de transações diárias
        df['QTDE_TRANSACOES_DIA'] = ts['BLOCK_DATE'].count()
        
        # Média de GAS diário
        df['MEDIA_GAS_DIA'] = ts['GAS'].mean()
        
        # Média de GAS_LIMIT diário
        df['MEDIA_GAS_LIMIT_DIA'] = ts['RECEIPT_GAS_USED'].mean()
        
        # Quantidade de compradores únicos diários
        df['QTDE_COMPRADORES_UNICOS_DIA'] = ts['FROM_ADDRESS'].nunique()
        
        # Quantidade de vendedores únicos diários
        df['QTDE_VENDEDORES_UNICOS_DIA'] = ts['TO_ADDRESS'].nunique()
        
        # Cálculo de NEW_HOLDER
        df['NEW_HOLDER'] = (ts['FROM_ADDRESS'].nunique() + ts['TO_ADDRESS'].nunique()) / ts['BLOCK_DATE'].count()
        
        # Transforma a coluna 'BLOCK_DATE' em índice temporal
        df.set_index('BLOCK_DATE', inplace=True)
        
        # Adicionar o DataFrame criado à lista correspondente
        ts_nft_list.append(df)



Comum DataFrames:

grouped_df_Comum 1

Blue Chip DataFrames:

grouped_df_Blue Chip 1

grouped_df_Blue Chip 2

grouped_df_Blue Chip 3

grouped_df_Blue Chip 4

grouped_df_Blue Chip 5

grouped_df_Blue Chip 6

grouped_df_Blue Chip 7

grouped_df_Blue Chip 8

grouped_df_Blue Chip 9

grouped_df_Blue Chip 10

grouped_df_Blue Chip 11

grouped_df_Blue Chip 12

grouped_df_Blue Chip 13

grouped_df_Blue Chip 14

grouped_df_Blue Chip 15

grouped_df_Blue Chip 16

grouped_df_Blue Chip 17

grouped_df_Blue Chip 18

grouped_df_Blue Chip 19

grouped_df_Blue Chip 20


In [19]:
# Verifique os 5 primeiros registros de ts_nft_comum
print("5 primeiros registros de ts_nft_comum:")
for i, df in enumerate(ts_nft_comum):
    print(f"\nDataFrame {i+1} (ts_nft_comum):")
    print(df.head(5))

# Verifique os 5 primeiros registros de ts_nft_blue
print("\n5 primeiros registros de ts_nft_blue:")
for i, df in enumerate(ts_nft_blue):
    print(f"\nDataFrame {i+1} (ts_nft_blue):")
    print(df.head(5))


5 primeiros registros de ts_nft_comum:

DataFrame 1 (ts_nft_comum):
                               QTDE_TRANSACOES_DIA  MEDIA_GAS_DIA  \
BLOCK_DATE                                                          
2023-08-29 00:00:00.000000000                 2300  438149.763043   
2023-08-30 00:00:00.000000256                 1964  452742.823829   
2023-08-31 00:00:00.000000000                 2685  254344.509125   
2023-09-01 00:00:00.000000000                 1357  699083.578482   
2023-09-02 00:00:00.000000000                   89  541240.730337   

                               MEDIA_GAS_LIMIT_DIA  \
BLOCK_DATE                                           
2023-08-29 00:00:00.000000000        412754.713043   
2023-08-30 00:00:00.000000256        366573.197047   
2023-08-31 00:00:00.000000000        214897.683799   
2023-09-01 00:00:00.000000000        454693.144436   
2023-09-02 00:00:00.000000000        426413.842697   

                               QTDE_COMPRADORES_UNICOS_DIA  \
BLOCK_D

            QTDE_TRANSACOES_DIA  MEDIA_GAS_DIA  MEDIA_GAS_LIMIT_DIA  \
BLOCK_DATE                                                            
2021-12-12                10762  194996.549991        185230.735830   
2021-12-13                 8994  224468.732600        196201.918279   
2021-12-14                 2829  288286.507953        225110.428773   
2021-12-15                  931  273871.216971        224421.458647   
2021-12-16                  625  247666.100800        201143.707200   

            QTDE_COMPRADORES_UNICOS_DIA  QTDE_VENDEDORES_UNICOS_DIA  \
BLOCK_DATE                                                            
2021-12-12                          443                        3700   
2021-12-13                         1639                        3781   
2021-12-14                         1079                        1480   
2021-12-15                          479                         614   
2021-12-16                          288                         380   

    

In [20]:
# Função para normalizar uma lista de DataFrames
def normalize_dataframes(dataframe_list):
    # Inicialize o MinMaxScaler
    scaler = MinMaxScaler()

    for idx, ts in enumerate(dataframe_list):
        # Selecione apenas as colunas numéricas para normalização
        columns_to_normalize = [
            'QTDE_TRANSACOES_DIA',
            'MEDIA_GAS_DIA',
            'MEDIA_GAS_LIMIT_DIA',
            'QTDE_COMPRADORES_UNICOS_DIA',
            'QTDE_VENDEDORES_UNICOS_DIA',
            'NEW_HOLDER'
        ]

        # Aplique o Min-Max Scaling às colunas selecionadas
        dataframe_list[idx][columns_to_normalize] = scaler.fit_transform(ts[columns_to_normalize])



In [21]:
# Função para imprimir informações e os 5 primeiros registros de uma lista de DataFrames
def print_info_and_head(dataframe_list):
    for i, df in enumerate(dataframe_list):
        print(f"\nDataFrame {i+1}:")
        print("Informações após a normalização:")
        print(df.info())
        print("5 primeiros registros após a normalização:")
        print(df.head())


In [22]:
# Normalizar ts_nft_comum
normalize_dataframes(ts_nft_comum)

# Normalizar ts_nft_blue
normalize_dataframes(ts_nft_blue)


In [23]:
# Imprimir informações e os 5 primeiros registros de ts_nft_blue após a normalização
print("Informações e 5 primeiros registros de ts_nft_blue após a normalização:")
print_info_and_head(ts_nft_blue)

print("\n=======================================================================================\n")


# Imprimir informações e os 5 primeiros registros de ts_nft_comum após a normalização
print("Informações e 5 primeiros registros de ts_nft_comum após a normalização:")
print_info_and_head(ts_nft_comum)

Informações e 5 primeiros registros de ts_nft_blue após a normalização:

DataFrame 1:
Informações após a normalização:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 433 entries, 2022-01-06 00:00:00 to 2023-03-14 00:00:00
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   QTDE_TRANSACOES_DIA          433 non-null    float64
 1   MEDIA_GAS_DIA                433 non-null    float64
 2   MEDIA_GAS_LIMIT_DIA          433 non-null    float64
 3   QTDE_COMPRADORES_UNICOS_DIA  433 non-null    float64
 4   QTDE_VENDEDORES_UNICOS_DIA   433 non-null    float64
 5   NEW_HOLDER                   433 non-null    float64
dtypes: float64(6)
memory usage: 23.7 KB
None
5 primeiros registros após a normalização:
            QTDE_TRANSACOES_DIA  MEDIA_GAS_DIA  MEDIA_GAS_LIMIT_DIA  \
BLOCK_DATE                                                            
2022-01-06             0.733482       0.06603

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 681 entries, 2021-05-03 00:00:00 to 2023-03-14 00:00:00
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   QTDE_TRANSACOES_DIA          681 non-null    float64
 1   MEDIA_GAS_DIA                681 non-null    float64
 2   MEDIA_GAS_LIMIT_DIA          681 non-null    float64
 3   QTDE_COMPRADORES_UNICOS_DIA  681 non-null    float64
 4   QTDE_VENDEDORES_UNICOS_DIA   681 non-null    float64
 5   NEW_HOLDER                   681 non-null    float64
dtypes: float64(6)
memory usage: 37.2 KB
None
5 primeiros registros após a normalização:
            QTDE_TRANSACOES_DIA  MEDIA_GAS_DIA  MEDIA_GAS_LIMIT_DIA  \
BLOCK_DATE                                                            
2021-05-03             1.000000       0.045517             0.014220   
2021-05-04             0.264403       0.158777             0.125355   
2021-05-05            

## Divisão do dados do Dataframe Blue Chips normalizados 

Divisão do DataFrame (normalized_dataset) em conjuntos de treinamento, validação e teste usando a função train_test_split do scikit-learn.

* train_test_split(normalized_dataset, test_size=0.4, random_state=42): Divisão do DataFrame **normalized_dataset** em dois conjuntos, o conjunto de **treinamento (train_data)** e o conjunto **temporário (temp_data)**. O hiperparâmetro test_size=0.4 define que 40% dos dados serão usados para o conjunto de teste e, consequentemente, 60% dos dados restantes serão usados para o conjunto de treinamento. O random_state=42 é usado para garantir que a divisão dos dados seja reproduzível.

* train_test_split(temp_data, test_size=0.5, random_state=42): Divisão do conjunto **temporário (temp_data)** em conjuntos de **validação (validation_data)** e **teste (test_data)**. O hiperparâmetro test_size=0.5 define que 50% dos dados serão usados para o conjunto de teste e, portanto, os outros 50% serão usados para o conjunto de validação. Novamente, o random_state=42 é usado para garantir que a divisão seja reproduzível.

Após essas operações, teremmos os seguintes conjuntos de dados:

* **train_data**: Conjunto de treinamento, que representa 60% dos dados originais.
* **validation_data**: Conjunto de validação, que representa 20% dos dados originais.
* **test_data**: Conjunto de teste, que também representa 20% dos dados originais.

Esses conjuntos podem ser usados para **treinar e testar** os modelos de detecção de anomalias (por exemplo, LOF) e, posteriormente, avaliar o desempenho do modelo usando as métricas adequadas, como **precisão, recall, F1-score** e outras métricas relevantes para o problema específico de detecção de anomalias.

In [24]:
# Dividir cada DataFrame em ts_nft_blue em conjuntos de treinamento, validação e teste
train_data_blue = []
validation_data_blue = []
test_data_blue = []

for df in ts_nft_blue:
    # Dividir o DataFrame em treinamento (60%), validação (20%) e teste (20%)
    train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)
    validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
    
    # Adicionar os DataFrames resultantes às listas correspondentes
    train_data_blue.append(train_df)
    validation_data_blue.append(validation_df)
    test_data_blue.append(test_df)


## Criação de modelos de detecção de anomalias e comitê de avaliação:

* Três modelos de detecção de anomalias são definidos:
    LOF, IF e OC-SVM. Esses modelos são inicializados com diferentes configurações.

* Treinamento dos modelos de detecção de anomalias:
    Os modelos LOF, IF e OC-SVM são treinados nos dados de treinamento (train_df).

* Previsões usando os modelos:
    Após o treinamento, os modelos fazem previsões nos dados de validação (validation_df) e teste (test_df). As previsões são médias das pontuações de anomalia (decision_function) calculadas por cada modelo.

* Cálculo da média das médias das previsões:
    As médias das previsões dos modelos LOF, IF e OC-SVM são somadas e divididas por 3 para obter as previsões de um ensemble. Essas previsões ensemble são armazenadas nas variáveis ensemble_validation_predictions e ensemble_test_predictions.

* Definição do limiar (threshold) para classificar anomalias:
    Um limiar (threshold) de 0.5 é definido para classificar as anomalias. Qualquer valor acima desse limiar será considerado uma anomalia.

* Aplicação do limiar para classificar anomalias:
    O código aplica o limiar às previsões ensemble e converte os valores acima do limiar em 1 (anomalias) e os valores abaixo do limiar em 0 (não anomalias). Essas previsões finais são armazenadas nas variáveis final_validation_predictions, final_test_predictions e final_predictions.

* Exibição das previsões:
    O código imprime as previsões do ensemble para os conjuntos de validação, teste e conjunto de dados a ser avaliado.



In [25]:
def decode_predictions(predictions):
    decoded_predictions = []
    for prediction in predictions:
        if prediction == 1:
            decoded_predictions.append("Anomalia")
        else:
            decoded_predictions.append("Não Anomalia")
    return decoded_predictions

# Exemplo de uso da função com a lista final_validation_predictions
# final_validation_predictions = [1, 0, 1, 1, 0, 0, 1]  # Exemplo de lista de previsões
# decoded_predictions = decode_predictions(final_validation_predictions)

# for i, prediction in enumerate(decoded_predictions):
#    print(f"Dado {i + 1}: {prediction}")


In [26]:
# Definição e criação de modelos para detecção de anomalias
lof = LocalOutlierFactor(novelty=True)
isolation_forest = IsolationForest(contamination=0.1)
one_class_svm = OneClassSVM(nu=0.1)

In [27]:
# Listas para armazenar as médias das previsões
validation_pred_lof = []
validation_pred_if = []
validation_pred_ocsvm = []

test_pred_lof = []
test_pred_if = []
test_pred_ocsvm = []

test_pred_lof_comum = []
test_pred_if_comum = []
test_pred_ocsvm_comum = []

In [28]:
# Loop através das listas de DataFrames
for train_df, validation_df, test_df in zip(train_data_blue, validation_data_blue, test_data_blue):

    # Treine os modelos de detecção de anomalias nos dados de treinamento
    lof.fit(train_df)
    isolation_forest.fit(train_df)
    one_class_svm.fit(train_df)

    # Faça previsões usando os modelos individuais nos dados de validação
    validation_pred_lof.append(np.mean(lof.decision_function(validation_df)))
    validation_pred_if.append(np.mean(isolation_forest.decision_function(validation_df)))
    validation_pred_ocsvm.append(np.mean(one_class_svm.decision_function(validation_df)))

    # Faça previsões usando os modelos nos dados de teste
    test_pred_lof.append(np.mean(lof.decision_function(test_df)))
    test_pred_if.append(np.mean(isolation_forest.decision_function(test_df)))
    test_pred_ocsvm.append(np.mean(one_class_svm.decision_function(test_df)))

    
# Definição do limiar (threshold) para classificar as anomalias
threshold = 0.5

# Calculo da média das previsões de cada modelo para o conjunto de dados de validação
ensemble_validation_predictions = (np.array(validation_pred_lof) + np.array(validation_pred_if) + np.array(validation_pred_ocsvm)) / 3

# Aplicar o limiar para classificar anomalias (valores acima do limiar são considerados anomalias)
final_validation_predictions = (ensemble_validation_predictions > threshold).astype(int)

# Exibir as previsões para o conjunto de dados de Validação
print("Previsões do Ensemble (Validação):")
print(final_validation_predictions)
decoded_predictions = decode_predictions(final_validation_predictions)

for i, prediction in enumerate(decoded_predictions):
    print(f"Conjunto de Dados {i + 1}: {prediction}")
    
# Calculo da média das previsões de cada modelo para o conjunto de dados de teste
ensemble_test_predictions = (np.array(test_pred_lof) + np.array(test_pred_if) + np.array(test_pred_ocsvm)) / 3

# Aplicar o limiar para classificar anomalias (valores acima do limiar são considerados anomalias)
final_test_predictions = (ensemble_test_predictions > threshold).astype(int)

# Exiba as previsões para o conjunto de dados de teste
print("\nPrevisões do Ensemble (Teste):")
print(final_test_predictions)
decoded_predictions = decode_predictions(final_test_predictions)

for i, prediction in enumerate(decoded_predictions):
    print(f"Conjunto de Dados {i + 1}: {prediction}")

Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature name

Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature name

Previsões do Ensemble (Validação):
[0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0]
Conjunto de Dados 1: Não Anomalia
Conjunto de Dados 2: Não Anomalia
Conjunto de Dados 3: Anomalia
Conjunto de Dados 4: Não Anomalia
Conjunto de Dados 5: Não Anomalia
Conjunto de Dados 6: Anomalia
Conjunto de Dados 7: Não Anomalia
Conjunto de Dados 8: Não Anomalia
Conjunto de Dados 9: Anomalia
Conjunto de Dados 10: Não Anomalia
Conjunto de Dados 11: Não Anomalia
Conjunto de Dados 12: Não Anomalia
Conjunto de Dados 13: Anomalia
Conjunto de Dados 14: Não Anomalia
Conjunto de Dados 15: Não Anomalia
Conjunto de Dados 16: Não Anomalia
Conjunto de Dados 17: Não Anomalia
Conjunto de Dados 18: Não Anomalia
Conjunto de Dados 19: Anomalia
Conjunto de Dados 20: Não Anomalia

Previsões do Ensemble (Teste):
[0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
Conjunto de Dados 1: Não Anomalia
Conjunto de Dados 2: Não Anomalia
Conjunto de Dados 3: Anomalia
Conjunto de Dados 4: Não Anomalia
Conjunto de Dados 5: Não Anomalia
Conjunto de 

Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


In [29]:
# Loop através das listas de DataFrames
      
for train_df in train_data_blue:

    # Treine os modelos de detecção de anomalias nos dados de treinamento
    lof.fit(train_df)
    isolation_forest.fit(train_df)
    one_class_svm.fit(train_df)

for test_df_comum in ts_nft_comum:

    # Faça previsões usando os modelos nos dados de teste
    test_pred_lof_comum.append(np.mean(lof.decision_function(test_df_comum)))
    test_pred_if_comum.append(np.mean(isolation_forest.decision_function(test_df_comum)))
    test_pred_ocsvm_comum.append(np.mean(one_class_svm.decision_function(test_df_comum)))
    

ensemble_predictions = np.array([])

# Definição do limiar (threshold) para classificar as anomalias
threshold = 0.5  

ensemble_predictions = (np.array(test_pred_lof_comum) + np.array(test_pred_if_comum) + np.array(test_pred_ocsvm_comum)) / 3
#print(ensemble_predictions)

# Aplicar o limiar para classificar anomalias (valores acima do limiar são considerados anomalias)
final_predictions = (ensemble_predictions > threshold).astype(int)

# Exiba as previsões para os dados de teste
print("Previsões do Ensemble (NFT Comum):")
print(final_predictions)
decoded_predictions = decode_predictions(final_predictions)

for i, prediction in enumerate(decoded_predictions):
    print(f"Conjunto de Dados {i + 1}: {prediction}")
    

    


Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warni

Previsões do Ensemble (NFT Comum):
[0]
Conjunto de Dados 1: Não Anomalia


Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(
